In [9]:
import pandas as pd
import numpy as np
from multicore import *
from data_preprocess import DataPreprocessor
from data_util import *

In [10]:
data_name = 'ciao'
u_core = 5
i_core = 5

In [3]:
if data_name == 'ml-1m':
    data_path = '../org_data/'+data_name+'/ratings.dat'
    data = pd.read_csv(data_path, sep='::',
                names=['userId', 'itemId', 'rating', 'timestamp'],
                dtype={'userId':np.int32,'itemId':np.int32,'rating':np.float64,'timestamp':np.int32},
                engine='python')
if data_name == 'ml-20m':
    data_path = '../org_data/'+data_name+'/ratings.csv'
    data = pd.read_csv(data_path, sep=',',
#                 names=['userId', 'itemId', 'rating', 'timestamp'],
#                 dtype={'userId':np.int32,'itemId':np.int32,'rating':np.float64,'timestamp':np.int32},
                engine='python')
if data_name == 'ml-100k':
    data_path = '../org_data/'+data_name+'/u.data'
    data = pd.read_csv(data_path, sep='\t',
                names=['userId', 'itemId', 'rating', 'timestamp'],
                dtype={'userId':np.int32,'itemId':np.int32,'rating':np.float64,'timestamp':np.int32},
                engine='python')
if data_name == 'cd':
    data_path = '../org_data/'+data_name+'/cd.csv'
    data = pd.read_csv(data_path, sep=',',
                names=['userId', 'itemId', 'rating', 'timestamp'],
                dtype={'userId':np.int32,'itemId':np.int32,'rating':np.float64,'timestamp':np.int32},
                engine='python')
if data_name == 'ciao' or data_name == 'epinions':
    data_path = '../org_data/'+data_name+'/rating_with_timestamp.txt'
    data = pd.read_csv(data_path, sep='  ',
                names=['userId', 'itemId', 'category','rating', 'helpfulness','timestamp'],
                dtype={'userId':np.int32,'category':np.int32,'itemId':np.int32,'rating':np.float64,'helpfulness':np.int32,'timestamp':np.int32},
                engine='python')    


In [4]:
data_ = data[['userId','itemId','rating','timestamp']]

In [5]:
len(data_)

36065

In [6]:
filtered_data = run_multicore(data_.to_numpy(), '../data/'+data_name, u_core, i_core)
filtered_data = np.stack(filtered_data)
print(len(filtered_data))

100%|██████████| 10307/10307 [00:00<00:00, 402329.35it/s]

Filtering 5_5-core data
Iteration 1
Number of removed record: 23227
Iteration 2
Number of removed record: 1738
Iteration 3
Number of removed record: 550
Iteration 4
Number of removed record: 176
Iteration 5
Number of removed record: 55
Iteration 6
Number of removed record: 12
Iteration 7
Number of removed record: 0
Save data to "../data/ciao/multicore_5_5.csv"
10307


In [7]:
len(np.unique(filtered_data[:,0])),max(filtered_data[:,0]),len(np.unique(filtered_data[:,1])),max(filtered_data[:,1])
# len(np.unique(filtered_data[:,1])), max(filtered_data[:,1])

(814, 813.0, 980, 979.0)

In [11]:
datapath = '../data/'+data_name+'/multicore_'+str(u_core)+'_'+str(i_core)+'.csv'
# print(datapath)
dg = DataPreprocessor(data_name, datapath)

31it [00:00, 302.43it/s]

total num of users: 814
total num of items: 980
start generating user history...


814it [00:01, 414.39it/s]


In [12]:
train_filename = '../data/'+data_name+'/train_data.csv'
test_filename = '../data/'+data_name+'/test_data.csv'
dg.write_csv(train_filename, test_filename)

691 691


In [13]:
get_orginal_data(data_name)

done!


In [14]:
datapath = '../data/'+data_name+'/train_data_org.csv'
data = pd.read_csv(datapath,names=['userId', 'itemId', 'rating'],
                     dtype={'userId':np.int32,'itemId':np.int32,'rating':np.float64})
data.head()

,userId,itemId,rating
0,0,12,5.0
1,0,11,4.0
2,0,10,4.0
3,0,9,5.0
4,0,8,5.0


In [16]:
items = data['itemId'].to_list()
items = np.array(items)
idx, nb = np.unique(items, return_counts=True)
# idx,nb

In [ ]:
pivot_point = 0
while np.sum(nb>pivot_point)/dg.nb_item >= 0.2:
    pivot_point += 1
print('pivot point:', pivot_point)

cost_indicator = np.zeros(dg.nb_item)
item_exposure = np.zeros(dg.nb_item)
for i in range(len(nb)):
    item_exposure[idx[i]] = nb[i]
    if nb[i] > pivot_point:
        cost_indicator[idx[i]] += 1

In [ ]:
np.save('../data/'+data_name+'/item_cost_indicator_28.npy', cost_indicator)

In [ ]:
np.save('../data/'+data_name+'/item_exposure_28.npy', item_exposure)

In [ ]:
pivot_point = 0
while np.sum(item_exposure>pivot_point)/len(item_exposure) >= 0.2:
    pivot_point += 1

cost_indicator = np.zeros(len(item_exposure))
for i in range(len(item_exposure)):
    if item_exposure[i] > pivot_point:
        cost_indicator[i] += 1

In [ ]:
sum(cost_indicator)

In [ ]:
len(item_exposure)